In [1]:
!nvidia-smi

Sat May 22 10:32:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.48       Driver Version: 451.48       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   45C    P8     3W /  N/A |    398MiB /  4096MiB |     18%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [2]:
!python train.py --epochs 20 --lr 0.02 --save --summarize

Namespace(batch_size=512, epochs='20', lr=0.02, plot_graph=False, save=True, summarize=True, total=25)


In [3]:
!python inference.py --total 25